In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score,brier_score_loss
import numpy as np

from lifelines import CoxPHFitter
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

# set up data and task name

In [ ]:
path=''
task = ''

# 5-fold corss validation

In [ ]:
data = pd.read_parquet(path)
data['year'] = data.study_entry.apply(lambda x: x.year)

In [ ]:
def generate_data(data):
    train, test = train_test_split(data, test_size=0.3)
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    return train, test


def train_iter(train, test):
    time_col = 'time2event'
    label_col = 'label'
    formular = "age+gen_ethnicity+imd2015_5+systolic+systolic_std+BMI+hdl+smoke+chd_history+diabetes+rheumatoid_arthritis+atrial_fibrillation+ckd+migraine+lupus_erythematosus+mental_ill+hiv_aids+ED+antihtn+antipsychotic+corticosteroid"
    t0 = 12*5
    
    cph = CoxPHFitter()
    cph.fit(train, duration_col=time_col, event_col=label_col, formula=formular)

    auroc, auprc = evaluate_auroc_auprc(cph, test, t0=t0)
    
    return auroc, auprc

def evaluate_auroc_auprc(model, data, t0=12*5):
    def ccl(p):
        return np.log(-np.log(1 - p))

    T = model.duration_col
    E = model.event_col
    t0 = t0

    predictions_at_t0 = np.clip(1 - model.predict_survival_function(data, times=[t0]).T.squeeze(), 1e-10, 1 - 1e-10)
    
    auprc = average_precision_score(data[E].values, predictions_at_t0)
    auroc = roc_auc_score(data[E].values, predictions_at_t0)
    
    return auroc, auprc

def prediction(model, data, t0=12*5):
    def ccl(p):
        return np.log(-np.log(1 - p))

    T = model.duration_col
    E = model.event_col
    t0 = t0

    predictions_at_t0 = np.clip(1 - model.predict_survival_function(data, times=[t0]).T.squeeze(), 1e-10, 1 - 1e-10)
    
    patid = data.patid.values
    
    return patid, data[E].values, predictions_at_t0

In [ ]:
prc_list = []
roc_list = []
for i in range(5):
    train, test= generate_data(data)
    roc, prc = train_iter(train, test)
    print('iter {}, auroc: {}, auprc: {}'.format(i, roc, prc))
    prc_list.append(prc)
    roc_list.append(roc)

print('summary roc mean: {} 95%CI: {}'.format(np.mean(roc_list), 1.96*np.std(roc_list)))
print('summary prc mean: {} 95%CI: {}'.format(np.mean(prc_list), 1.96*np.std(prc_list)))

# cross region validation

In [ ]:
data = pd.read_parquet(path)

In [ ]:
# year
def generate_data(data, region):
    train, test = data[data['region'].isin(region[0])], data[data['region'].isin(region[1])]
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    return train, test

def train_iter(train, test):
    time_col = 'time2event'
    label_col = 'label'
    
    #
    formular = ""
    
    t0 = 12*5
    
    cph = CoxPHFitter()
    cph.fit(train, duration_col=time_col, event_col=label_col, formula=formular)

    auroc, auprc = evaluate_auroc_auprc(cph, test, t0=t0)
    
    return auroc, auprc

In [ ]:
region = [['4','5','6','7','8','9','10'], ['1','2','3']]

prc_list = []
roc_list = []
for i in range(5):
    train, test = generate_data(data, region)
    roc, prc = train_iter(train, test)

    prc_list.append(prc)
    roc_list.append(roc)

print('summary roc mean: {} 95%CI: {}'.format(np.mean(roc_list), 1.96*np.std(roc_list)))
print('summary prc mean: {} 95%CI: {}'.format(np.mean(prc_list), 1.96*np.std(prc_list)))

# OOD validation year

In [ ]:
data = pd.read_parquet(path)
data['year'] = data.study_entry.apply(lambda x: x.year)

In [ ]:
def generate_train_data(data, year=2000):
    train = data[data['year']< year]
    train = train.reset_index(drop=True)
    return train

def train_model(train):
    time_col = 'time2event'
    label_col = 'label'
    
    # set up formular to train CPH model
    formular = ""
    t0 = 12*5
    
    cph = CoxPHFitter()
    cph.fit(train, duration_col=time_col, event_col=label_col, formula=formular)
    return cph

def generate_test_data(data, year=(2000, 2001)):
    test = data[(data['year']>= year[0])&(data['year']<year[1])]
    test = test.reset_index(drop=True)
    return test

def eval_model(test, clf, t0=12*5):
    auroc, auprc = evaluate_auroc_auprc(clf, test, t0=t0)
    
    return auroc

## Discrimination

In [ ]:

year = [(1999, 2000), (2000, 2001), (2001, 2002), (2002, 2003), (2003, 2004), (2004, 2005), 
        (2005, 2006), (2006, 2007), (2007, 2008), (2008, 2009), (2009, 2010)]

train = generate_train_data(data, year=2000)
clf = train_model(train)

for each in year:
    test = generate_test_data(data, year=each)
    roc= eval_model(test, clf)
    print('{}-{} ROC: {}'.format(each[0], each[1], roc))

In [ ]:
year = [(2001, (2006,2007)), (2002, (2007,2008)), (2003, (2008,2009)), (2004, (2009,2010))]


for each in year:
    train= generate_train_data(data, year=each[0])
    clf = train_model(train)
    
    test= generate_test_data(data, year=each[1])
    roc = eval_model(test, clf)
    print('{},{}-{} ROC: {}'.format(each[0],each[1][0], each[1][1], roc,))

## Calibration

In [ ]:
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_parquet(path)
    
year = [(2000, 2001), (2001, 2002), (2002, 2003), (2003, 2004), (2004, 2005), 
        (2005, 2006), (2006, 2007), (2007, 2008), (2008, 2009), (2009, 2010)]

train= generate_train_data(data, year=2000)
clf = train_model(train)
plt.figure(figsize=(2, 2), dpi=300)
for each in year:
    test = generate_test_data(data, year=each)
    _, label, predict = prediction( clf, test)
    
    observe, estimate = calibration_curve(label, predict, n_bins=10)
    calibration_df = pd.DataFrame({'Observed risk': observe, 'Estimated risk': estimate})
    sns.regplot(x="Estimated risk", y="Observed risk", data=calibration_df, order=2, ci=None, label='{}-{}'.format(each[0], each[1])  , marker='')
x= np.linspace(0,1,10)
plt.plot(x,x,label='reference',color='black')
plt.title(task)

In [ ]:
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_parquet(path)

name_map = {
    '2000': "A",
    '2001': "B",
    '2002': "C",
    '2003': "D", 
    '2004': "E"
}


year = [(2000, (2005,2006)), (2001, (2006,2007)), (2002, (2007,2008)), (2003, (2008,2009)), (2004, (2009,2010))]

plt.figure(figsize=(2, 2), dpi=300)
for each in year:
    train = generate_train_data(data, year=each[0])
    clf = train_model(train)
    test = generate_test_data(data, year=each[1])
    _, label, predict = prediction(clf, test)
    
    observe, estimate = calibration_curve(label, predict, n_bins=10)
    calibration_df = pd.DataFrame({'Observed risk': observe, 'Estimated risk': estimate})
    sns.regplot(x="Estimated risk", y="Observed risk", data=calibration_df, order=2, ci=None, label=name_map.get(each[0]) , marker='')
x= np.linspace(0,1,10)
plt.plot(x,x,label='reference',color='black')
plt.title(task)